In [ ]:
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("attribs_adder", CombinedAttributesAdder()),
    ("std_scaler", StandardScaler())
])

# num_pipeline.set_params(attribs_adder__add_bedrooms_per_room=False)
# num_pipeline.fit_transform(housing_num)

In [ ]:
num_attribs = list(housing_num)
cat_attribs = list(housing["ocean_proximity"].unique())

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", "passthrough", cat_attribs)
])

cat_attribs

# full_pipeline.set_params(num__attribs_adder__add_bedrooms_per_room=True)
# housing_prepared_2 = full_pipeline.fit_transform(housing)

['INLAND', 'NEAR OCEAN', '<1H OCEAN', 'NEAR BAY', 'ISLAND']

In [ ]:
forest_reg = RandomForestRegressor(random_state=42)

final_pipeline = Pipeline([
    ("transformations", full_pipeline),
    ("model", forest_reg)
])

# final_pipeline.set_params(model__n_estimators = 2)
# final_pipeline.fit(housing, housing_labels)

In [ ]:
one_hot_encoder = OneHotEncoder()
housing_treated = one_hot_encoder.fit_transform(housing[["ocean_proximity"]])

housing[one_hot_encoder.categories_[0]] = housing_treated.toarray() 

In [ ]:
housing_ready = housing.drop("ocean_proximity", axis=1)
housing_ready

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
12655,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,72100.0,0.0,1.0,0.0,0.0,0.0
15502,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,279600.0,0.0,0.0,0.0,0.0,1.0
2908,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750,82700.0,0.0,1.0,0.0,0.0,0.0
14053,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,112500.0,0.0,0.0,0.0,0.0,1.0
20496,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,238300.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15174,-117.07,33.03,14.0,6665.0,1231.0,2026.0,1001.0,5.0900,268500.0,1.0,0.0,0.0,0.0,0.0
12661,-121.42,38.51,15.0,7901.0,1422.0,4769.0,1418.0,2.8139,90400.0,0.0,1.0,0.0,0.0,0.0
19263,-122.72,38.44,48.0,707.0,166.0,458.0,172.0,3.1797,140400.0,1.0,0.0,0.0,0.0,0.0
19140,-122.70,38.31,14.0,3155.0,580.0,1208.0,501.0,4.1964,258100.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
# housing_prepared[0]

In [ ]:
# full_pipeline.fit_transform(housing_ready)[0]

In [ ]:
param_grid = [
    {"model__n_estimators": [3, 10, 30], 'model__max_features': [2, 4, 6, 8], "transformations__num__attribs_adder__add_bedrooms_per_room": [False]},
    # {'model__bootstrap': [False], 'model__n_estimators': [3, 10], 'model__max_features': [2, 3, 4]}
]


grid_search = GridSearchCV(final_pipeline, param_grid, cv=5, scoring="neg_mean_squared_error", return_train_score = True)

grid_search.fit(housing_ready, housing_labels)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('transformations',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('attribs_adder',
                                                                                          CombinedAttributesAdder()),
                                                                                         ('std_scaler',
                                                                                          StandardScaler())]),
                                                                         ['longitude',
                                                                          'latitude',
   

In [ ]:
grid_search.best_params_

{'model__max_features': 6,
 'model__n_estimators': 30,
 'transformations__num__attribs_adder__add_bedrooms_per_room': False}

In [ ]:
grid_search.best_estimator_.named_steps["transformations"].transform(housing_ready).shape[1]

15

In [ ]:
grid_search.feature_names_in_

array(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', '<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY',
       'NEAR OCEAN'], dtype=object)

In [ ]:
grid_search.n_features_in_

14

In [ ]:
cvres = grid_search.cv_results_
for mean_test_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_test_score), params)

63575.9739672717 {'model__max_features': 2, 'model__n_estimators': 3, 'transformations__num__attribs_adder__add_bedrooms_per_room': False}
55934.295008652516 {'model__max_features': 2, 'model__n_estimators': 10, 'transformations__num__attribs_adder__add_bedrooms_per_room': False}
52528.2390678022 {'model__max_features': 2, 'model__n_estimators': 30, 'transformations__num__attribs_adder__add_bedrooms_per_room': False}
58874.541629771884 {'model__max_features': 4, 'model__n_estimators': 3, 'transformations__num__attribs_adder__add_bedrooms_per_room': False}
51945.43067586159 {'model__max_features': 4, 'model__n_estimators': 10, 'transformations__num__attribs_adder__add_bedrooms_per_room': False}
49420.54023703712 {'model__max_features': 4, 'model__n_estimators': 30, 'transformations__num__attribs_adder__add_bedrooms_per_room': False}
58791.25871270213 {'model__max_features': 6, 'model__n_estimators': 3, 'transformations__num__attribs_adder__add_bedrooms_per_room': False}
51656.9318014044

In [ ]:
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("attribs_adder", CombinedAttributesAdder()),
    ("std_scaler", StandardScaler())
])

# num_pipeline.set_params(attribs_adder__add_bedrooms_per_room=False)
# num_pipeline.fit_transform(housing_num)

In [ ]:
num_attribs = list(housing_num)
cat_attribs = list(housing["ocean_proximity"].unique())

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", "passthrough", cat_attribs)
])

cat_attribs

# full_pipeline.set_params(num__attribs_adder__add_bedrooms_per_room=True)
# housing_prepared_2 = full_pipeline.fit_transform(housing)

['INLAND', 'NEAR OCEAN', '<1H OCEAN', 'NEAR BAY', 'ISLAND']

In [ ]:
forest_reg = RandomForestRegressor(random_state=42)

final_pipeline = Pipeline([
    ("transformations", full_pipeline),
    ("model", forest_reg)
])

# final_pipeline.set_params(model__n_estimators = 2)
# final_pipeline.fit(housing, housing_labels)

In [ ]:
one_hot_encoder = OneHotEncoder()
housing_treated = one_hot_encoder.fit_transform(housing[["ocean_proximity"]])

housing[one_hot_encoder.categories_[0]] = housing_treated.toarray() 

In [ ]:
housing_ready = housing.drop("ocean_proximity", axis=1)
housing_ready

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
12655,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,72100.0,0.0,1.0,0.0,0.0,0.0
15502,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,279600.0,0.0,0.0,0.0,0.0,1.0
2908,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750,82700.0,0.0,1.0,0.0,0.0,0.0
14053,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,112500.0,0.0,0.0,0.0,0.0,1.0
20496,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,238300.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15174,-117.07,33.03,14.0,6665.0,1231.0,2026.0,1001.0,5.0900,268500.0,1.0,0.0,0.0,0.0,0.0
12661,-121.42,38.51,15.0,7901.0,1422.0,4769.0,1418.0,2.8139,90400.0,0.0,1.0,0.0,0.0,0.0
19263,-122.72,38.44,48.0,707.0,166.0,458.0,172.0,3.1797,140400.0,1.0,0.0,0.0,0.0,0.0
19140,-122.70,38.31,14.0,3155.0,580.0,1208.0,501.0,4.1964,258100.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
# housing_prepared[0]

In [ ]:
# full_pipeline.fit_transform(housing_ready)[0]

In [ ]:
param_grid = [
    {"model__n_estimators": [3, 10, 30], 'model__max_features': [2, 4, 6, 8], "transformations__num__attribs_adder__add_bedrooms_per_room": [False]},
    # {'model__bootstrap': [False], 'model__n_estimators': [3, 10], 'model__max_features': [2, 3, 4]}
]


grid_search = GridSearchCV(final_pipeline, param_grid, cv=5, scoring="neg_mean_squared_error", return_train_score = True)

grid_search.fit(housing_ready, housing_labels)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('transformations',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('attribs_adder',
                                                                                          CombinedAttributesAdder()),
                                                                                         ('std_scaler',
                                                                                          StandardScaler())]),
                                                                         ['longitude',
                                                                          'latitude',
   

In [ ]:
grid_search.best_params_

{'model__max_features': 6,
 'model__n_estimators': 30,
 'transformations__num__attribs_adder__add_bedrooms_per_room': False}

In [ ]:
grid_search.best_estimator_.named_steps["transformations"].transform(housing_ready).shape[1]

15

In [ ]:
grid_search.feature_names_in_

array(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', '<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY',
       'NEAR OCEAN'], dtype=object)

In [ ]:
grid_search.n_features_in_

14

In [ ]:
cvres = grid_search.cv_results_
for mean_test_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_test_score), params)

63575.9739672717 {'model__max_features': 2, 'model__n_estimators': 3, 'transformations__num__attribs_adder__add_bedrooms_per_room': False}
55934.295008652516 {'model__max_features': 2, 'model__n_estimators': 10, 'transformations__num__attribs_adder__add_bedrooms_per_room': False}
52528.2390678022 {'model__max_features': 2, 'model__n_estimators': 30, 'transformations__num__attribs_adder__add_bedrooms_per_room': False}
58874.541629771884 {'model__max_features': 4, 'model__n_estimators': 3, 'transformations__num__attribs_adder__add_bedrooms_per_room': False}
51945.43067586159 {'model__max_features': 4, 'model__n_estimators': 10, 'transformations__num__attribs_adder__add_bedrooms_per_room': False}
49420.54023703712 {'model__max_features': 4, 'model__n_estimators': 30, 'transformations__num__attribs_adder__add_bedrooms_per_room': False}
58791.25871270213 {'model__max_features': 6, 'model__n_estimators': 3, 'transformations__num__attribs_adder__add_bedrooms_per_room': False}
51656.9318014044

In [ ]:
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("attribs_adder", CombinedAttributesAdder()),
    ("std_scaler", StandardScaler())
])

# num_pipeline.set_params(attribs_adder__add_bedrooms_per_room=False)
# num_pipeline.fit_transform(housing_num)

In [ ]:
num_attribs = list(housing_num)
cat_attribs = list(housing["ocean_proximity"].unique())

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", "passthrough", cat_attribs)
])

cat_attribs

# full_pipeline.set_params(num__attribs_adder__add_bedrooms_per_room=True)
# housing_prepared_2 = full_pipeline.fit_transform(housing)

['INLAND', 'NEAR OCEAN', '<1H OCEAN', 'NEAR BAY', 'ISLAND']

In [ ]:
forest_reg = RandomForestRegressor(random_state=42)

final_pipeline = Pipeline([
    ("transformations", full_pipeline),
    ("model", forest_reg)
])

# final_pipeline.set_params(model__n_estimators = 2)
# final_pipeline.fit(housing, housing_labels)

In [ ]:
one_hot_encoder = OneHotEncoder()
housing_treated = one_hot_encoder.fit_transform(housing[["ocean_proximity"]])

housing[one_hot_encoder.categories_[0]] = housing_treated.toarray() 

In [ ]:
housing_ready = housing.drop("ocean_proximity", axis=1)
housing_ready

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
12655,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,72100.0,0.0,1.0,0.0,0.0,0.0
15502,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,279600.0,0.0,0.0,0.0,0.0,1.0
2908,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750,82700.0,0.0,1.0,0.0,0.0,0.0
14053,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,112500.0,0.0,0.0,0.0,0.0,1.0
20496,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,238300.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15174,-117.07,33.03,14.0,6665.0,1231.0,2026.0,1001.0,5.0900,268500.0,1.0,0.0,0.0,0.0,0.0
12661,-121.42,38.51,15.0,7901.0,1422.0,4769.0,1418.0,2.8139,90400.0,0.0,1.0,0.0,0.0,0.0
19263,-122.72,38.44,48.0,707.0,166.0,458.0,172.0,3.1797,140400.0,1.0,0.0,0.0,0.0,0.0
19140,-122.70,38.31,14.0,3155.0,580.0,1208.0,501.0,4.1964,258100.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
# housing_prepared[0]

In [ ]:
# full_pipeline.fit_transform(housing_ready)[0]

In [ ]:
param_grid = [
    {"model__n_estimators": [3, 10, 30], 'model__max_features': [2, 4, 6, 8], "transformations__num__attribs_adder__add_bedrooms_per_room": [False]},
    # {'model__bootstrap': [False], 'model__n_estimators': [3, 10], 'model__max_features': [2, 3, 4]}
]


grid_search = GridSearchCV(final_pipeline, param_grid, cv=5, scoring="neg_mean_squared_error", return_train_score = True)

grid_search.fit(housing_ready, housing_labels)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('transformations',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('attribs_adder',
                                                                                          CombinedAttributesAdder()),
                                                                                         ('std_scaler',
                                                                                          StandardScaler())]),
                                                                         ['longitude',
                                                                          'latitude',
   

In [ ]:
grid_search.best_params_

{'model__max_features': 6,
 'model__n_estimators': 30,
 'transformations__num__attribs_adder__add_bedrooms_per_room': False}

In [ ]:
grid_search.best_estimator_.named_steps["transformations"].transform(housing_ready).shape[1]

15

In [ ]:
grid_search.feature_names_in_

array(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', '<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY',
       'NEAR OCEAN'], dtype=object)

In [ ]:
grid_search.n_features_in_

14

In [ ]:
cvres = grid_search.cv_results_
for mean_test_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_test_score), params)

63575.9739672717 {'model__max_features': 2, 'model__n_estimators': 3, 'transformations__num__attribs_adder__add_bedrooms_per_room': False}
55934.295008652516 {'model__max_features': 2, 'model__n_estimators': 10, 'transformations__num__attribs_adder__add_bedrooms_per_room': False}
52528.2390678022 {'model__max_features': 2, 'model__n_estimators': 30, 'transformations__num__attribs_adder__add_bedrooms_per_room': False}
58874.541629771884 {'model__max_features': 4, 'model__n_estimators': 3, 'transformations__num__attribs_adder__add_bedrooms_per_room': False}
51945.43067586159 {'model__max_features': 4, 'model__n_estimators': 10, 'transformations__num__attribs_adder__add_bedrooms_per_room': False}
49420.54023703712 {'model__max_features': 4, 'model__n_estimators': 30, 'transformations__num__attribs_adder__add_bedrooms_per_room': False}
58791.25871270213 {'model__max_features': 6, 'model__n_estimators': 3, 'transformations__num__attribs_adder__add_bedrooms_per_room': False}
51656.9318014044